# Operational Wind Lidar Processing with LROSE

## About NCAR Integrated Sounding System (ISS)

The ISS is a suite of instruments for profiling the atmosphere from ground level. Instrumentation we operate includes wind profiling radars, scanning wind lidars, sounding systems, and more.

### Scanning wind lidar
We currently own a Leosphere WindCube WLS300S scanning wind lidar that we deploy for many projects. 

### Operational data products
[pic of lidar dataflow diagram]

Our automated processing pipeline consists of these steps:
1. Convert from Leosphere netCDF format (almost CFRadial compliant, but not quite) to CFRadial netCDF
2. Create single-scan plot files from CFRadial netCDF
3. Calculate VAD winds from CFRadial netCDF RHI scans
4. Calculate consensus averages from the VAD winds

### Initialize python

### Convert Leosphere format files to CFRadial

We use the LROSE RadxConvert tool for this.

### Generate plots for each CFRadial PPI or RHI file

We use HawkEye from LROSE for this.

### Generate VAD winds for each RHI file

We use python code in the iss-lidar repo for this.

### Generate consensus averaged winds from VAD winds

We use python code in the iss-lidar repo for this.